# Included Modules and Packages

In [41]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

# Supporting code

In [42]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [43]:
session = driver.session(database="neo4j")

In [44]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [45]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [46]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [47]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [48]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [49]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [50]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [51]:
cursor = connection.cursor()

In [52]:
my_api_key = "************"

gmaps.configure(api_key=my_api_key)

# Wipe out the Neo4j database

Call the function my_neo4j_wipe_out_database() to wipe out the Neo4j database

In [53]:
my_neo4j_wipe_out_database()

In [54]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Query the list of stations and create station nodes. For our case the relationships are undirectional, so we don't differentiate "arrival" and "departure"




In [55]:
connection.rollback()

query = """

select station,line from lines
order by station,line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station, line= row
    
    my_neo4j_create_node(line +' '+ station)

In [56]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 0
-------------------------


# Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight




In [57]:
connection.rollback()

query = """

select 
    l_from.station,
    l_from.line as from_line,
    l_to.line as to_line,
    stations.transfer_time
from 
lines as l_from 
    left join lines as l_to
        on l_from.station=l_to.station
    left join stations
        on stations.station=l_from.station
where l_from.line <> l_to.line
order by station, from_line, to_line

"""

df=pd.read_sql(query,connection)

df.tail()

,station,from_line,to_line,transfer_time
203,West Oakland,red,green,283.0
204,West Oakland,red,yellow,283.0
205,West Oakland,yellow,blue,283.0
206,West Oakland,yellow,green,283.0
207,West Oakland,yellow,red,283.0


In [58]:
for i in df.index:

    station, from_line,to_line,transfer_time= df.loc[i,:]
    
    my_neo4j_create_relationship_one_way(from_line+' '+station, to_line+' '+station,int(transfer_time))

In [59]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 208
-------------------------


# Query the list of all segments between each station and its adjoining stations, create a relationship for each segment

In [60]:
connection.rollback()

query = """

select 
    l_from.line,
    l_from.station as from_station,
    l_to.station as to_station,
    travel_times.travel_time
from 
lines as l_from 
    inner join lines as l_to
        on (l_from.line=l_to.line) and (l_from.sequence=l_to.sequence-1)
    left join travel_times
        on ((travel_times.station_1=l_from.station) and (travel_times.station_2=l_to.station)) 
            or ((travel_times.station_2=l_from.station) and (travel_times.station_1=l_to.station))
order by line, from_station, to_station

"""

df=pd.read_sql(query,connection)

df.tail()

,line,from_station,to_station,travel_time
103,yellow,Rockridge,MacArthur,240.0
104,yellow,San Bruno,SFO,240.0
105,yellow,South San Francisco,San Bruno,240.0
106,yellow,Walnut Creek,Lafayette,300.0
107,yellow,West Oakland,Embarcadero,420.0


In [61]:
for i in df.index:

    line, from_station,to_station,travel_time= df.loc[i,:]
    
    my_neo4j_create_relationship_two_way(line+' '+from_station, line+' '+to_station,int(travel_time))

In [62]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 424
-------------------------


# Build the graph

In [63]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

# Run Closeness Centrality Algo

In [64]:
# Closeness Centrality: 
query = """

CALL gds.beta.closeness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as closeness
ORDER BY score DESC

"""

closeness=my_neo4j_run_query_pandas(query)

closeness.head()

,name,closeness
0,yellow West Oakland,0.147520
1,green West Oakland,0.147327
2,red West Oakland,0.146753
3,blue West Oakland,0.146563
4,yellow Embarcadero,0.143038


# Query the stations' coordinates

In [65]:
connection.rollback()

query = """

select * from stations

"""

station_coord=pd.read_sql(query,connection)

station_coord.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282.0
1,16th Street Mission,37.764847,-122.420042,287.0
2,19th Street,37.807869,-122.268980,67.0
3,24th Street Mission,37.752000,-122.418700,277.0
4,Antioch,37.996281,-121.783404,0.0


# Join the closeness and locations of stations

In [66]:
closeness['station']=[i.split(' ',1)[1] for i in closeness['name']]

closeness=closeness.merge(station_coord)

closeness.head()

,name,closeness,station,latitude,longitude,transfer_time
0,yellow West Oakland,0.147520,West Oakland,37.804900,-122.295100,283.0
1,green West Oakland,0.147327,West Oakland,37.804900,-122.295100,283.0
2,red West Oakland,0.146753,West Oakland,37.804900,-122.295100,283.0
3,blue West Oakland,0.146563,West Oakland,37.804900,-122.295100,283.0
4,yellow Embarcadero,0.143038,Embarcadero,37.793056,-122.397222,304.0


# Demostrate the closeness centrality of stations with heatmap on Gmap

In [67]:
west_oakland=(37.804900,-122.295100)

In [74]:
figure_layout = {
    'width': '400px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=west_oakland,map_type = 'HYBRID', zoom_level=9,layout=figure_layout,)

heatmap_layer = gmaps.heatmap_layer(
    closeness[['latitude','longitude']],
    weights=closeness['closeness'],
    point_radius=10,
    gradient = ['gray', 'white','red']
)

fig.add_layer(heatmap_layer)
# fig.add_layer(gmaps.transit_layer())
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='400px'))

# Run Louvain Algo to group the stations

In [69]:
query = """

CALL gds.louvain.stream('ds_graph',{ relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

station_group=my_neo4j_run_query_pandas(query)

In [70]:
station_group

,name,community,intermediate_community
0,orange 19th Street,16,None
1,orange Ashby,16,None
2,orange Downtown Berkeley,16,None
3,orange MacArthur,16,None
4,red 19th Street,16,None
...,...,...,...
109,red Embarcadero,112,None
110,red West Oakland,112,None
111,yellow 12th Street,112,None
112,yellow Embarcadero,112,None


# Demostrate groups with different color on Gmap

In [71]:
station_group=station_group.merge(closeness)

station_group['color']=np.nan

station_group['community'].value_counts().shape

for i,j in zip(set(station_group['community']),['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52', 'teal','silver']):
    station_group.loc[station_group['community']==i,'color']=j

In [73]:
fig = gmaps.figure(center=west_oakland,zoom_level=9,layout=figure_layout)

group_symbols = gmaps.symbol_layer(
    station_group[['latitude','longitude']], 
    fill_color=station_group['color'].to_list(), 
    stroke_color=station_group['color'].to_list(), 
    scale=6,
)

fig.add_layer(group_symbols)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='400px'))